# SIDA: Speaker Identification for Archives

In [1]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio
import pandas as pd

!mkdir -p /sharedfolder/sida_classifier/

os.chdir('/sharedfolder/sida_classifier/')

## Download audio files for training
#### (You may want to comment out the lines below once the download is complete.)
#!wget -N http://xtra.arloproject.com/datasets/audio/The_World_WGBH_100_episodes.zip
#!unzip The_World_WGBH_100_episodes.zip

training_audio_dir_name = "The_World_WGBH_100_episodes"

In [2]:
## Download new 1-second labels

csv_url = "https://raw.githubusercontent.com/hipstas/aapb-speaker-labels/master/speaker_labels_randomized/The_World_WGBH_labels_100_episodes.csv"

csv_string = urllib2.urlopen(csv_url)

train_table_df = pd.read_csv(csv_url)

train_table_df.head()

,Media file basename,Start time,Duration,Label,Labeled by
0,cpb-aacip-15-zg6g15v54b__20131015TW_,3212.0,1.0,Male,Stephen R. McLaughlin
1,cpb-aacip-500-r785p23b,346.0,1.0,Female,Stephen R. McLaughlin
2,cpb-aacip-15-9jq0st8f__240747_20130619TW,451.0,1.0,Male,Stephen R. McLaughlin
3,cpb-aacip-15-ns0ks6jt4h__20130117TW_,2127.0,1.0,Female,Stephen R. McLaughlin
4,cpb-aacip-15-3n20c4t30z__20130925TW_,294.0,1.0,Female,Stephen R. McLaughlin


In [3]:
## Choosing variables to extract and assigning variables we'll use below

labels_to_use = ['Marco Werman', 'Male', 'Female']

media_dir_pathname = '/sharedfolder/sida_classifier/' + training_audio_dir_name

class_dir_pathname = '/sharedfolder/sida_classifier/_classes_' + training_audio_dir_name

In [ ]:
%%capture
## Excerpting WAV clips corresponding to labels
#### (This may take a while.)

os.chdir('/sharedfolder/sida_classifier/')

train_table_groups = train_table_df.groupby(['Media file basename', 'Label'])

for name, group in train_table_groups:
    list_of_lists = group.values.tolist()
    basename, start, duration, label, labeled_by = list_of_lists[0]
    filename = str(basename) + '.mp3'
    media_path = os.path.join(media_dir_pathname, filename)
    label_dir_pathname = str(os.path.join(class_dir_pathname, label.replace(' ','_')))
    if label in labels_to_use:
        subclip_pairs = []
        for row in list_of_lists:
            basename, start, duration, label, labeled_by = row
            subclip_pairs.append((float(start), float(duration)))
        try: subprocess.call(['mkdir', '-p', label_dir_pathname])
        except: pass
        try:
            attk.subclip_list(media_path, subclip_pairs, label_dir_pathname)
        except Exception as e: 
            print(e)

In [ ]:
%%capture
## Extract vowel segments from labeled audio clips
#### (This may take a while.)

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    try:
        attk.batch_extract_vowels(dir_name)
    except Exception as e:
        print("ERROR: " + dir_name)
        print(e)

In [ ]:
%%capture
## Extract features (MFCCs, deltas, and delta-deltas) from Marco Werman & UBM vowel clips, then write features to CSVs

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name, '_vowel_clips'))
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

In [ ]:
%%capture
## Download and unzip prepared UBM feature set

os.chdir('/sharedfolder/sida_classifier/')

!wget -q https://github.com/hipstas/aapb-ubm/blob/master/UBM_feature_set/AAPB_female_vowel_mfccs_and_deltas.zip?raw=true -O AAPB_female_vowel_mfccs_and_deltas.zip
!wget -q https://github.com/hipstas/aapb-ubm/blob/master/UBM_feature_set/AAPB_male_vowel_mfccs_and_deltas.zip?raw=true -O AAPB_male_vowel_mfccs_and_deltas.zip

!unzip AAPB_female_vowel_mfccs_and_deltas.zip
!unzip AAPB_male_vowel_mfccs_and_deltas.zip

In [ ]:
## Continue to the next notebook to train and run the speaker ID classifier.